# 예제 9.11 SIFT 특징 검출 및 디스크립터

In [1]:
import cv2
import numpy as np

In [2]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x1-x2) ** 2 + (y1-y2) ** 2)

In [3]:
def filteringByDistance(kp, distE = 0.5):
    size = len(kp)
    mask = np.arange(1, size + 1).astype(np.bool_)
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    print("False 개수", list(mask).count(False), "-> 필터링을 통해서 제거되는 pt의 개수")
    np_kp = np.array(kp)
    return list(np_kp[mask])

In [4]:
src = np.zeros(shape = (512,512,3), dtype = np.uint8) + 255
cv2.rectangle(src, (100,100), (300,300), (0,0,0), -1)
cv2.rectangle(src, (200,200), (400,400), (0,0,0), -1)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

In [5]:
siftF = cv2.SIFT_create(edgeThreshold = 80)
kp = siftF.detect(gray)
len(kp)

12

In [6]:
kp = sorted(kp, key=lambda f:f.response, reverse = True)
filtered_kp = filteringByDistance(kp, 10)
len(filtered_kp)

False 개수 5 -> 필터링을 통해서 제거되는 pt의 개수


7

In [7]:
kp, des = siftF.compute(gray, filtered_kp)
des.shape, des.dtype, des[0]

((7, 128),
 dtype('float32'),
 array([  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 170.,  22.,   0.,
          0.,   0.,   0.,   0.,   0., 174.,  45.,   0.,   0.,   0.,   0.,
          0.,   0.,  16.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,  76.,  10., 143.,  11.,   0.,   0.,
          0.,   0.,  86.,  54., 174.,  34.,   0.,   0.,   0.,   0.,  10.,
         56.,  31.,   3.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0., 174.,  51.,  16.,   0.,   0.,   0.,   0.,
          0., 174., 123.,  97.,   1.,   0.,   0.,   0.,   0.,  85., 161.,
         12.,   0.,   0.,   0.,   0.,   0.,   0.,   6.,   0.,   0.,   0.,
          0.,   0.,   0.,  57.,   7.,   0.,   0.,   0.,   0.,   0.,   0.,
         47.,   8.,   0.,   0.,   0.,   0.,   0.,   0.,  13.,   8.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.], dtype=float32))

In [8]:
dst = cv2.drawKeypoints(src, filtered_kp, None, color=(0,0,255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x,y), (size,size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst, [box], True, (0,255,0), 2)
    cv2.circle(dst, (round(x), round(y)), round(f.size/2), (255,0,0), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()